# 初始化
引入必备模块和包，创建ArgumentParser对象，用于解析命令行参数

In [ ]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchdiffeq import odeint_adjoint as odeint
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import imageio

# 确定ODE的数值方法
method = 'dopri5'
# 指定数据集大小
data_size = 1000
batch_time = 10
batch_size = 20
# 指定训练循环的迭代次数
niters = 2000
test_freq = 20


# 环境检查
设置了一个ODE求解环境，包括选择求解器、计算设备、初始条件、评估时间点以及系统的动态参数。这些设置都是为了在接下来的步骤中进行ODE求解准备的.

In [ ]:
true_y0 = torch.tensor([[2., 0.]])# 初始化真实的初始条件，可以看到这里的情况是一个1x2的张量
t = torch.linspace(0., 25., data_size)# 创建时间点的张量，可能用于在这些时间点上评估ODE的解
true_A = torch.tensor([[-0.1, 2.0], [-2.0, -0.1]])# 定义ODE的系数矩阵，代表系统的动态行为

# 生成批量训练数据

In [ ]:
# Lambda 类代表了一个常微分方程的动态行为
class Lambda(nn.Module):
# 定义了forward方法，它指定了ODE的具体形式。对于每个时间点t和状态y，forward方法返回y的三次方与系数矩阵true_A的矩阵乘积
    def forward(self, t, y):
        return torch.mm(y**3, true_A)

# 无梯度下求解ODE
with torch.no_grad():
# 注意，这里的odeint需要一个模型，odeint需要一个模型（此处为Lambda()实例）
# （接上）一个初始状态（true_y0），一组时间点（t），以及一个求解器方法（此处为dopri5，即显式Runge-Kutta方法的一种）  
    true_y = odeint(Lambda(), true_y0, t, method='dopri5')

# 生成批量训练数据
# 这些步骤使得此代码段能够在训练神经网络模型时提供批量的输入数据和标签
def get_batch():
    s = torch.from_numpy(np.random.choice(np.arange(data_size - batch_time, dtype=np.int64), batch_size, replace=False))
    batch_y0 = true_y[s]  # (M, D)
    batch_t = t[:batch_time]  # (T)
    batch_y = torch.stack([true_y[s + i] for i in range(batch_time)], dim=0)  # (T, M, D)
    return batch_y0, batch_t, batch_y

true_y

# 可视化

In [ ]:

def visualize(true_y, pred_y, odefunc, itr):

    # Trajectories
    plt.figure(1)
    plt.clf()
    plt.title('Trajectories')
    plt.xlabel('t')
    plt.ylabel('x,y')
    plt.plot(t.cpu().numpy(), true_y.cpu().numpy()[:, 0, 0], 'g-', label='True')
    plt.plot(t.cpu().numpy(), true_y.cpu().numpy()[:, 0, 1], 'g--', label='True')
    plt.plot(t.cpu().numpy(), pred_y.cpu().numpy()[:, 0, 0], 'b-', label='Predicted')
    plt.plot(t.cpu().numpy(), pred_y.cpu().numpy()[:, 0, 1], 'b--', label='Predicted')
    plt.xlim(t.cpu().min(), t.cpu().max())
    plt.ylim(-2, 2)
    plt.legend()

    # Phase Portrait
    plt.figure(2)
    plt.clf()
    plt.title('Phase Portrait')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.plot(true_y.cpu().numpy()[:, 0, 0], true_y.cpu().numpy()[:, 0, 1], 'g-', label='True')
    plt.plot(pred_y.cpu().numpy()[:, 0, 0], pred_y.cpu().numpy()[:, 0, 1], 'b-', label='Predicted')
    plt.xlim(-2, 2)
    plt.ylim(-2, 2)
    plt.legend()

    # Learned Vector Field
    plt.figure(3)
    plt.clf()
    plt.title('Learned Vector Field')
    plt.xlabel('x')
    plt.ylabel('y')

    y, x = np.mgrid[-2:2:21j, -2:2:21j]
    dydt = odefunc(0, torch.Tensor(np.stack([x, y], -1).reshape(21 * 21, 2))).cpu().detach().numpy()
    mag = np.sqrt(dydt[:, 0]**2 + dydt[:, 1]**2).reshape(-1, 1)
    dydt = (dydt / mag)
    dydt = dydt.reshape(21, 21, 2)

    # Animation
    def animate(i):
        plt.figure(1)
        plt.plot(t.cpu().numpy()[:i], true_y.cpu().numpy()[:i, 0, 0], 'g-', alpha=0.5)
        plt.plot(t.cpu().numpy()[:i], true_y.cpu().numpy()[:i, 0, 1], 'g--', alpha=0.5)
        plt.plot(t.cpu().numpy()[:i], pred_y.cpu().numpy()[:i, 0, 0], 'b-', alpha=0.5)
        plt.plot(t.cpu().numpy()[:i], pred_y.cpu().numpy()[:i, 0, 1], 'b--', alpha=0.5)

        plt.figure(2)
        plt.plot(true_y.cpu().numpy()[:i, 0, 0], true_y.cpu().numpy()[:i, 0, 1], 'g-', alpha=0.5)
        plt.plot(pred_y.cpu().numpy()[:i, 0, 0], pred_y.cpu().numpy()[:i, 0, 1], 'b-', alpha=0.5)

        return [plt.figure(j) for j in range(1, 4)]

    ani = animation.FuncAnimation(plt.gcf(), animate, frames=len(t), interval=100, blit=True)
    plt.show()


# 生成存储文件

In [ ]:
# 生成存储文件
def makedirs(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)


# 求解ODE

In [ ]:
# 设置神经网络
class ODEFunc(nn.Module):
    # 初始化
    def __init__(self):
        super(ODEFunc, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(2, 50), #线性层
            nn.Tanh(), # 激活层
            nn.Linear(50, 2), # 线性层
        )
        #权重和偏置的初始化
        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.1)
                nn.init.constant_(m.bias, val=0)
    # 前向传播
    def forward(self, t, y):
        return self.net(y**3)


In [ ]:
ii = 0

func = ODEFunc()
# 使用优化算法及学习率
optimizer = optim.RMSprop(func.parameters(), lr=1e-3)
end = time.time()
# 初始化时间记录器
# time_meter = RunningAverageMeter(0.97)
# 损失记录器
# loss_meter = RunningAverageMeter(0.97)

# 训练循环
for itr in range(1, niters + 1):
    # 梯度归零
    optimizer.zero_grad()
    batch_y0, batch_t, batch_y = get_batch()
    # 前向传播和损失计算
    pred_y = odeint(func, batch_y0, batch_t)
    loss = torch.mean(torch.abs(pred_y - batch_y))
    loss.backward()
    optimizer.step()

    # time_meter.update(time.time() - end)
    # loss_meter.update(loss.item())

    #定期评估和可视化
    if itr % test_freq == 0:
        with torch.no_grad():
            pred_y = odeint(func, true_y0, t)
            loss = torch.mean(torch.abs(pred_y - true_y))
            print('Iter {:04d} | Total Loss {:.6f}'.format(itr, loss.item()))
            visualize(true_y, pred_y, func, ii)
            ii += 1
    # 时间更新
    end = time.time()
